# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
from sympy import *

In [2]:
DEBUG = False

# Provided functions

In [3]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (n, 3)
            where each row denotes the parameters of the line equation
        centers: np.ndarray of shape (n, 3)
            where each row denotes the homogeneous coordinates of the centers
        lengths: np.ndarray of shape (n)
            where each row denotes the length of the line segment
        angles: np.ndarray of shape(n)
            where each row denotes the angle the line makes with the x-axis in radians
    """
    n = 0
    lines = []
    centers = []
    lengths = []
    angles = []

    np.set_printoptions(suppress=True)
    
    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines.append(line)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers.append(center)
        # Get length of the line segment
        length = np.linalg.norm(pt1[:2] - pt2[:2])
        lengths.append(length)
        # Get the angle of the segment (in radians)
        centered_p2 = pt2 - pt1
        angle = np.arctan2(centered_p2[1],centered_p2[0])%(2*math.pi)
        angles.append(angle)
        
        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
        
        if(DEBUG):
            print(n)
            print(np.array2string(np.array(lines), separator=', '))
            print(np.array2string(np.array(centers), separator=', '))
            print(np.array2string(np.array(lengths), separator=', '))
            print(np.array2string(np.array(angles), separator=', '))
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

        n += 1
        
    return n, np.array(lines), np.array(centers), np.array(lengths), np.array(angles)

In [4]:
def plot_lines_and_vp(im, lines, vp, path="vp.jpg"):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (n,3)
            where each row denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    x = vp[0] / vp[2]
    y = vp[1] / vp[2]
    
    bx1 = min(1, x) - 10
    bx2 = max(im.shape[1], x) + 10
    by1 = min(1, y) - 10
    by2 = max(im.shape[0], y) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[0]):
        if lines[i, 0] < lines[i, 1]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    plt.plot(x, y, 'ro')
    
    min_x = min(x, 0)
    max_x = max(x, im.shape[1])
    min_y = min(y, 0)
    max_y = max(y, im.shape[0])
    
    plt.xlim((min_x, max_x))
    plt.ylim((max_y, min_y))
    
    plt.show()
    plt.savefig(path)

In [20]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def angular_distance(theta_1, theta_2):
    """
    Solves for smallest angle between two angles in radians
    """
    big = max(theta_1, theta_2)
    small = min(theta_1, theta_2)
    if big - small < math.pi:
        return big - small
    return (small - big) % math.pi

def dist_point_line(x1, y1, a, b, c): 
    """
    Solves for orthogonal distance between point and line
    """
    return abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))

def get_vanishing_point(n, lines, centers, lengths, angles):
    """
    Solves for the vanishing point using the user-input lines.
    Solves by picking candidate with lowest sum of residuals 
        (distances between point and other lines)
    """
    candidates = []
    for i in range(n):
        for j in range(i,n):
            if(i != j):
                candidate = np.cross(lines[i], lines[j])
                candidates.append(candidate)
    best_score = float("inf")
    #best_score = 0
    best_candidate = []
    for candidate in candidates: # iterate over candidate points
#         score = 0
#         for i in range(n): # iterate over all lines
#             centered_pt = centers[i] - candidate 
#             alpha = np.arctan2(centered_pt[1],centered_pt[0])%(2*math.pi)
#             theta = angles[i]%(2*math.pi)
#             exp_term = -angular_distance(alpha,theta)/(2*(SIGMA**2))
#             sub_score = lengths[i] * np.exp(exp_term)
#             #sub_score = np.exp(exp_term)
#             score += sub_score
        score = 0
        for i in range(n): # iterate over all lines
            norm_candidate = (candidate[0] / candidate[2], candidate[1] / candidate[2])
            score += dist_point_line(norm_candidate[0], norm_candidate[1], lines[i][0], lines[i][1], lines[i][2])
        if(score < best_score):
            best_score = score
            best_candidate = candidate
    if(DEBUG):
        print(best_candidate)
        print(best_score)
        print("~~~~~~~~~~~~~")
    return best_candidate

In [7]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # horizon line is just line between points 0 and 1
    return np.cross(vpts[0], vpts[1])

In [8]:
def plot_horizon_line(im, vpts):
    """
    Plots the horizon line.
    """
    plt.figure()
    plt.imshow(im)
    pt1 = vpts[0][0] / vpts[0][2], vpts[0][1] / vpts[0][2]
    pt2 = vpts[1][0] / vpts[1][2], vpts[1][1] / vpts[1][2]
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')
    # plot the vanishing points
    for vp in vpts[:2]:
        plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.savefig('horizon_and_horizontal_vanishing_pts.png')
    plt.plot(vpts[2][0] / vpts[2][2], vpts[2][1] / vpts[2][2], 'ro')
    plt.savefig('horizon_and_all_vanishing_pts.png')
    
    # get horizon line and normalize parameters s.t. a**2 + b**2 = 1
    line = get_horizon_line(vpts)
    val = np.linalg.norm(line[:2])
    norm_line = line / val
    print("Normalized line parameters - a: " + str(norm_line[0]) + ", b: " + str(norm_line[1]) + ", c: " + str(norm_line[2]))
    
    return

In [9]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    """
    K = [[f, 0, u_0],
         [0, f, v_0],
         [0, 0, 1  ]]
         
    and for each pair of vanishing points v_i, v_j, i != j:
        v_i.T * K.inv.T * K.inv * v_j = 0
        
    First method uses my whiteboard solution
    
    Second solution uses SymPy
    """
    vpts_norm = (vpts.T / vpts[:,2]).T
    x0 = vpts_norm[0][0]
    y0 = vpts_norm[0][1]
    x1 = vpts_norm[1][0]
    y1 = vpts_norm[1][1]
    x2 = vpts_norm[2][0]
    y2 = vpts_norm[2][1]
    
    A0 = -(x0 + x1)
    A1 = -(x0 + x2)
    A2 = -(x1 + x2)
    B0 = -(y0 + y1)
    B1 = -(y0 + y2)
    B2 = -(y1 + y2)
    C0 = x0*x1 + y0*y1
    C1 = x0*x2 + y0*y2
    C2 = x1*x2 + y1*y2
    
    D = (B1-B0) / (A0-A1)
    E = (C1-C0) / (A0-A1)
    
    v = (A2*E + C2 - A1*E - C1) / (A1*D + B1 - A2*D - B2)
    u = ((B1 - B0)*v + (C1-C0)) / (A0-A1)
    f = (-(v**2 + u**2 + A0*v + B0*u + C0))**(0.5)
    
    if(DEBUG):
        f_, px, py = symbols('f_ px py')
        K = Matrix([[f_,0,px],[0,f_,py],[0,0,1]])
        vpts_m = Matrix(vpts_norm.tolist())
        eq1 = Eq((vpts_m.row(0) * K * vpts_m.row(1).T, 0))
        eq2 = Eq((vpts_m.row(0) * K * vpts_m.row(2).T, 0))
        eq3 = Eq((vpts_m.row(1) * K * vpts_m.row(2).T, 0))
        all_possible_solutions = solve((eq1,eq2,eq3),(f_,px,py))

        print("Second method")
        print(all_possible_solutions)
    
    print("f: " + str(f))
    print("u: " + str(u))
    print("v: " + str(v))
    return f, u, v

In [10]:
def get_rotation_matrix(f, u, v, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # these are currently V_x = V_left = vpts[0], V_y = V_right = vpts[1], V_z = V_vert = vpts[2]
    vpts_norm = (vpts.T / vpts[:,2]).T
    # convert so V_x = V_right, V_y = V_vert, V_z = V_left, aka 0,1,2 -> 1,2,0
    vpts_RVL = np.roll(vpts_norm, -1, axis=0)
    
    K = np.array([[f, 0., u], [0., f, v], [0., 0., 1.]])
    K_inv = np.linalg.inv(K)
    R = np.zeros((3,3))
    for i in range(3):
        R[:,i] = np.matmul(K_inv, vpts_RVL[i].T)
        
    print("Rotation Matrix (R): ")
    print(R)
    
    return R

In [17]:
def estimate_height(coords_person, coords_object, vpts, im, object_name, R=5.5):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    
    Assumes coordinates are from top to bottom
    """

    r = coords_person[:,0] # top of person
    b = coords_person[:,1] # bottom of person
    t0 = coords_object[:,0] # top of object
    b0 = coords_object[:,1] # bottom of object
    vpts_norm = (vpts.T / vpts[:,2]).T

    vx = vpts_norm[0] # left vanishing point 
    vy = vpts_norm[1] # right vanishing point
    vz = vpts_norm[2] # vertical vanishing point
    
    # compute v
    v = np.cross(np.cross(b, b0), np.cross(vx, vy))
    # normalize v
    v = v / v[2]
    # compute t
    t = np.cross(np.cross(v, t0), np.cross(r, b))
    # normalize t
    t = t / t[2]
    
    # plotting
    plt.figure()
    plt.imshow(im)
    
    # plot points/lines on horizon
    plt.plot(vx[0], vx[1], 'ro')  # v_x (left v point)
    plt.text(vx[0], vx[1], 'v_x', horizontalalignment='right') # v_x label
    plt.plot(vy[0], vy[1], 'ro')  # v_y (right v point)
    plt.text(vy[0], vy[1], 'v_y') # v_y label
    plt.plot(v[0], v[1], 'ro') # v (where line from person to object crosses vanishing point)
    plt.text(v[0], v[1], 'v', horizontalalignment='right')  # v label
    plt.plot((vx[0], vy[0]), (vx[1], vy[1]), 'g' ) # horizon line (line from vx to vy)
    
    # lines/points on person (reference object)
    plt.plot([t[0], b[0]], [t[1], b[1]], 'b')         # line from reference point t to bottom of person
    plt.plot(coords_person[0], coords_person[1], 'b') # line from top to bottom of person
    plt.text(((r + b) / 2)[0] + 10, ((r + b) / 2)[1], 'R') # person line label
    plt.plot(r[0], r[1], 'ro') # top of person
    plt.text(r[0], r[1], 'r')  # label point 'r'
    plt.plot(b[0], b[1], 'ro') # bottom of person
    plt.text(b[0], b[1], 'b')  # label point 'b'
    plt.plot(t[0], t[1], 'ro') # point 't' where line from person to object intersects persons' line
    plt.text(t[0], t[1], 't')  # lable point 't'
    
    # lines/points on object to measure
    plt.plot(coords_object[0], coords_object[1], 'b') # plot line representing object
    plt.text(((t0 + b0) / 2)[0] + 10, ((t0 + b0) / 2)[1], 'H')   # object height 'H' label
    plt.plot(t0[0], t0[1], 'ro')   # top of object
    plt.text(t0[0], t0[1], 't_0')  # label point 't_0'
    plt.plot(b0[0], b0[1], 'ro')   # bottom of object
    plt.text(b0[0], b0[1], 'b_0')  # label point 'b_0'
    
    # parallel lines orthogonal to object heights in the scene
    plt.plot((v[0], t[0]), (v[1], t[1]), 'b') # line from v to t
    plt.plot((v[0], b[0]), (v[1], b[1]), 'b') # line from v to b
    
    # save figure
    plt.savefig(object_name + str(".png"))
    
    if(DEBUG):
        print("b - coordinates of bottom of person: " + str(b))
        print("t - coordinates on person's vertical axis where the objects height intersects: " + str(t))

        print("length of t-b:")
        print(np.linalg.norm(t-b))
        print("length of r-b:")
        print(np.linalg.norm(r-b))
        print("ratio of ||t-b|| / ||r-b||")
        print(np.linalg.norm(t-b) / np.linalg.norm(r-b))
    
    # calculate height H
    H = R * (np.linalg.norm(t-b) / np.linalg.norm(r-b)) * (np.linalg.norm(vz-r)/np.linalg.norm(vz-t))
    return H

# Main function

In [12]:
# im = np.asarray(Image.open('CSL.jpeg'))
# num_vpts = 3
# vpts = np.zeros((num_vpts, 3))
# save_paths = ["vp_0.jpeg", "vp_1.jpeg", "vp_2.jpeg"]

# # Part 1
# # Get vanishing points for each of the directions
# for i in range(num_vpts):
#     print('Getting vanishing point %d' % i)
#     # Get at least three lines from user input
#     n, lines, centers, lengths, angles = get_input_lines(im)
#      # <YOUR IMPLEMENTATION> Solve for vanishing point
#     vp = get_vanishing_point(n, lines, centers, lengths, angles)
#     vpts[i] = vp
#     # Plot the lines and the vanishing point
#     plot_lines_and_vp(im, lines, vpts[i], save_paths[i])

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at lea

C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\4210025644.py:44: RuntimeWarning: invalid value encountered in double_scalars
  norm_candidate = (candidate[0] / candidate[2], candidate[1] / candidate[2])
C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\4210025644.py:44: RuntimeWarning: divide by zero encountered in double_scalars
  norm_candidate = (candidate[0] / candidate[2], candidate[1] / candidate[2])
C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\4210025644.py:15: RuntimeWarning: invalid value encountered in double_scalars
  return abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\2127262902.py:27: RuntimeWarning: divide by zero encountered in divide
  pt1 = pt1 / pt1[2]
C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\2127262902.py:27: RuntimeWarning: invalid value encountered in divide
  pt1 = pt1 / pt1[2]
C:\Users\oh_bo\AppData\Local\Temp\ipykernel_51112\2127262902.py:28: RuntimeWarning: divide by zero encountered 

In [13]:
# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, vpts)

Normalized line parameters - a: 0.012588235980863212, b: -0.9999207650183539, c: 214.77848822345968


In [14]:
# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(f, u, v, vpts)

f: 620.514469315845
u: 601.2836498070304
v: 319.8643011294356
Rotation Matrix (R): 
[[ 1.19735697 -0.12134167 -1.28540129]
 [-0.14205241  9.63852691 -0.17330844]
 [ 1.          1.          1.        ]]


In [22]:
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)
    
# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    
    print('Estimating height of %s' % obj)
    print("Assuming person is 5.5 feet")
    height = estimate_height(coords['person'], coords[obj], vpts, im, obj, 5.5)
    print('Height of %s is' % obj)
    print(str(height) + "ft")
    print('Estimating height of %s' % obj)
    print("Assuming person is 6 feet")
    height = estimate_height(coords['person'], coords[obj], vpts, im, obj, 6)
    print('Height of %s is' % obj)
    print(str(height) + "ft")
    print("~~~~~~~~~~~~~~~~~~~~~~~~")

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
Assuming person is 5.5 feet
Height of CSL building is
61.72710630092588ft
Estimating height of CSL building
Assuming person is 6 feet
Height of CSL building is
67.33866141919187ft
~~~~~~~~~~~~~~~~~~~~~~~~
Estimating height of the spike statue
Assuming person is 5.5 feet
Height of the spike statue is
39.358614876439894ft
Estimating height of the spike statue
Assuming person is 6 feet
Height of the spike statue is
42.936670774298065ft
~~~~~~~~~~~~~~~~~~~~~~~~
Estimating height of the lamp posts
Assuming person is 5.5 feet
Height of the lamp posts is
15.958236252334986ft
Estimating height of the la

In [16]:
# lines_3 = [
#     [[    38.10522262,    499.76465051, -97452.96284285],
#      [    14.65585485,    433.81330367, -88221.19009168],
#      [    -5.86234194,    441.14123109, -94851.86687787],
#      [  -133.36827917,    315.10087935, -98135.95385311],
#      [   -63.02017587,    265.27097285, -72357.46100873],
#      [   -61.55459038,    296.04826804, -75712.98038974],
#      [   -52.76107747,    312.16970838, -75940.77144161],
#      [  -203.71638246,    253.54628897, -94059.71942816],
#      [   -82.07278718,    159.7488179 , -52271.95680971],
#      [   -43.96756456,    196.38845504, -51553.81470098]],
#     [[    -27.84612422,     131.90269368,    6855.03008229],
#      [    -17.58702582,     108.45332592,   -1972.89821938],
#      [     -8.79351291,     109.9189114 ,  -14037.58644256],
#      [     -1.46558549,     109.9189114 ,  -23553.04240931],
#      [     46.89873553,     124.57476626,  -91392.3776603 ],
#      [     71.81368878,     211.04430989, -149349.05159911],
#      [     23.44936777,     121.64359528,  -59725.81436907],
#      [     54.22666296,     145.09296305, -112320.839288  ],
#      [     93.79747106,     152.42089048, -160248.46706709],
#      [     -4.39675646,      71.81368878,   -8402.96205266],
#      [     -5.86234194,      79.14161621,   -8003.63175948],
#      [     -4.39675646,    -106.98774043,   30476.69642556]],
#     [[  -159.7488179 ,     -4.39675646, 111605.39325263],
#      [   -63.02017587,     -1.46558549,  45102.74425739],
#      [  -177.33584373,     -5.86234194, 147574.60859283],
#      [  -131.90269368,     -7.32792743, 117975.11046922],
#      [   -95.26305655,     -4.39675646,  88449.90348814],
#      [   -45.43315005,     -4.39675646,  45365.97515681],
#      [  -106.98774043,      1.46558549,  57016.4712021 ],
#      [  -127.50593723,      2.93117097,  11721.30325917],
#      [   -74.74485975,      1.46558549,  17021.46933895],
#      [   -95.26305655,      4.39675646,  26000.8590287 ],
#      [   -92.33188558,     -7.32792743,  95734.90922507],
#      [  -117.24683883,     -2.93117097,  93594.84533247],
#      [  -117.24683883,     -2.93117097,  89642.63423309],
#      [  -108.45332592,      0.        ,  79157.07163106],
#      [  -108.45332592,      0.        ,  75978.11922504],
#      [  -101.12539849,      0.        ,  68028.51211233],
#      [  -120.1780098 ,      0.        ,  62880.1011868 ],
#      [  -133.36827917,      1.46558549,  62717.38748937],
#      [  -170.0079163 ,     11.72468388,   5177.16769827]]
# ]
# centers_3 = [
#     [[581.11531055, 150.68979867,   1.        ],
#      [561.3299065 , 184.39826483,   1.        ],
#      [563.52828473, 222.50348745,   1.        ],
#      [504.90486532, 525.14689018,   1.        ],
#      [698.36214938, 438.67734654,   1.        ],
#      [798.75475513, 421.82311346,   1.        ],
#      [674.91278162, 357.3373521 ,   1.        ],
#      [248.42740538, 570.58004022,   1.        ],
#      [458.00612978, 562.51932005,   1.        ],
#      [816.34178095, 445.27248123,   1.        ]],
#     [[894.01781168, 136.76673656,   1.        ],
#      [898.41456813, 163.88006804,   1.        ],
#      [891.81943345, 199.05411969,   1.        ],
#      [891.81943345, 226.16745116,   1.        ],
#      [812.67781724, 427.6854554 ,   1.        ],
#      [183.20885128, 645.32489998,   1.        ],
#      [ 62.29804874, 478.98094739,   1.        ],
#      [497.57693789, 588.16706605,   1.        ],
#      [832.46322129, 539.06995229,   1.        ],
#      [131.18056655, 125.04205268,   1.        ],
#      [134.84453026, 111.11899056,   1.        ],
#      [893.28501893, 248.15123345,   1.        ]],
#     [[ 682.24070904,  595.49499347,    1.        ],
#      [ 707.15566229,  366.86365776,    1.        ],
#      [ 825.13529387,  212.9771818 ,    1.        ],
#      [ 883.02592054,  204.91646163,    1.        ],
#      [ 919.66555767,  190.99339952,    1.        ],
#      [ 975.35780611,  239.35772053,    1.        ],
#      [ 535.68216051,  201.25249791,    1.        ],
#      [  95.27372216,  145.56024947,    1.        ],
#      [ 230.84037955,  158.75051884,    1.        ],
#      [ 293.86055542,  453.33320139,    1.        ],
#      [1003.20393034,  424.02149169,    1.        ],
#      [ 792.89241319,  215.17556002,    1.        ],
#      [ 759.18394702,  215.17556002,    1.        ],
#      [ 729.87223732,  215.17556002,    1.        ],
#      [ 700.56052761,  213.70997454,    1.        ],
#      [ 672.71440339,  211.51159631,    1.        ],
#      [ 523.22468388,  207.8476326 ,    1.        ],
#      [ 472.66198464,  218.83952374,    1.        ],
#      [  41.0470592 ,  153.62096964,    1.        ]]
# ]
# lengths_3 = [
#     [501.21523709, 434.06079818, 441.18018181, 342.16320968, 272.65405114,
#      302.37980192, 316.5969964 , 325.24772887, 179.59851675, 201.25002362],
#     [134.80996712, 109.87004769, 110.27009093, 109.92868154, 133.11034438,
#      222.92803017, 123.88315916, 154.89512227, 178.96953213,  71.94815747,
#       79.35844296, 107.07804663],
#     [159.80931227,  63.03721526, 177.43271548, 132.1060904 ,  95.36446618,
#       45.64540054, 106.99777822, 127.5396244 ,  74.75922686,  95.36446618,
#       92.62221988, 117.28347274, 117.28347274, 108.45332592, 108.45332592,
#      101.12539849, 120.1780098 , 133.37633159, 170.41173615]
# ]
# angles_3 = [
#     [6.20708621, 6.24941437, 0.01328825, 0.40039238, 0.23324517, 0.20499998, 
#      0.16743178, 0.67685213, 0.47459608, 0.22024872],
#     [0.20805614, 0.16076274, 0.07982999, 0.01333254, 5.92312598, 5.95519783,
#      6.09275019, 5.92552206, 5.73153032, 0.06114816, 0.07393904, 3.10051987],
#     [1.59831232, 1.59404795, 1.60384214, 1.62629483, 1.61691744, 1.6672701 ,
#      1.55709855, 1.54781187, 1.551191  , 1.52467521, 1.6499954 , 1.59579112,
#      1.59579112, 1.57079633, 1.57079633, 1.57079633, 1.57079633, 1.55980776,
#      1.50193984]
# ]
# n_3 = [len(row) for row in lines_3]

# im = np.asarray(Image.open('CSL.jpeg'))
# num_vpts = 3
# vpts = np.zeros((num_vpts, 3))
# save_paths = ["vp_0.jpeg", "vp_1.jpeg", "vp_2.jpeg"]

# for i in range(num_vpts):
#     n = n_3[i]
#     lines = np.array(lines_3[i])
#     centers = np.array(centers_3[i])
#     lengths = np.array(lengths_3[i])
#     angles = np.array(angles_3[i])
   
#     # <YOUR IMPLEMENTATION> Solve for vanishing point
#     vp = get_vanishing_point(n, lines, centers, lengths, angles)
#     vpts[i] = vp
#     # Plot the lines and the vanishing point
#     plot_lines_and_vp(im, lines, vpts[i], save_paths[i])